# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [2]:
from collections import Counter

import nltk
import numpy as np

In [3]:
class Batcher:
    def __init__(self, batch_size=1024, vocab_size=4000, window_size=2):
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__word2Ind = None
        self.__ind2Word = None
        self.__vocab = None
        
        self.__batch_size = batch_size
        self.__vocab_size = vocab_size - 1
        self.__window_size = window_size
        
    def __read_corpus(self, path):
        with open(path) as txt:
            corpus = txt.read()
        return corpus
    
    def __preprocess_corpus(self, corpus):
        """
        params:
            corpus - string
            
        returns: 
            preprocessed list of words in corpus saving order
        """
        return [word.strip().lower() for word in corpus.strip().split(' ') if word.strip().lower() not in self.__stopwords]
        
    def __make_vocab(self, corpus):
        word_counts = Counter(corpus)
        self.__vocab = set([x[0] for x in word_counts.most_common(self.__vocab_size)])
        
    def __make_idxs(self):
        self.__word2Ind = {'<UNK>':self.__vocab_size}
        self.__ind2Word = {self.__vocab_size:'<UNK>'}
        for i, word in enumerate(self.__vocab):
            self.__word2Ind[word] = i
            self.__ind2Word[i] = word
            
    def translate(self, indices):
        return [self.__ind2Word[idx] for idx in indices]
    
    def make_batches(self, path, method='skip-gram'):
        """
        params:
            method: one of ['skip-gram', 'cbow']
            
        returns:
            X, y matrices
        """
        corpus = self.__read_corpus(path)
        preproc_corpus = self.__preprocess_corpus(corpus)
        self.__make_vocab(preproc_corpus)
        self.__make_idxs()
        
        X, y = [], []
        padding_corpus = ['<PAD>' for i in range(self.__window_size)]   
        preproc_corpus = padding_corpus + preproc_corpus + padding_corpus.copy()
        for i in range(self.__window_size, len(preproc_corpus) - self.__window_size):
            central_idx, context_idx = None, []
            central, context = preproc_corpus[i], (preproc_corpus[i-self.__window_size:i] + preproc_corpus[i+1:i+self.__window_size+1])
            for word in context:
                if word in self.__vocab:
                    context_idx.append(self.__word2Ind[word])
                else:
                    context_idx.append(self.__vocab_size)
                
            if central in self.__vocab:
                central_idx = self.__word2Ind[central]
            else:
                central_idx = self.__vocab_size
            
            if method == 'cbow':
                X.append(np.array(context_idx))
                y.append(central_idx)
            elif method == 'skip-gram':
                X.append(central_idx)
                y.append(context_idx)
                
            if (i + 1) % self.__batch_size == 0:
                yield np.array(X), np.array(y)
                X, y = [], []
                
        yield np.array(X), np.array(y)

In [5]:
path = '/kaggle/input/text-data/text8'
trunc_path = '/kaggle/input/trunc-data/text8'
window_size = 2

batcher = Batcher(window_size=window_size)
for X, y in batcher.make_batches(trunc_path):
    print([batcher.translate([X[0]]), batcher.translate(y[0])])

[['anarchism'], ['<UNK>', '<UNK>', 'originated', 'term']]
[['communists'], ['subsequent', 'anarchist', 'include', 'emma']]
[['nine'], ['late', 'one', 'seven', 'zero']]
[['anarchism'], ['form', 'revolutionary', 'critical', 'formal']]
[['ideas'], ['anarchist', '<UNK>', 'dutch', 'punk']]
[['<UNK>'], ['prefer', 'consistent', 'environments', 'may']]
[['alternative'], ['attempt', '<UNK>', 'modes', 'communication']]
[['autism'], ['recent', 'increases', 'recognition', 'new']]
[['notably'], ['surface', 'composition', '<UNK>', 'content']]
[['six'], ['country', 'august', 'one', 'nine']]
[['code'], ['<UNK>', '<UNK>', 'capital', 'one']]
[['towards'], ['general', '<UNK>', 'mississippi', 'river']]
[['achilles'], ['burial', 'winner', 'rejected', 'arrangement']]
[['people'], ['war', 'resources', 'fought', 'trojan']]
[['elected'], ['combat', 'wrote', '<UNK>', 'success']]
[['dubbed'], ['campaign', 'lincoln', 'rail', '<UNK>']]
[['lincoln'], ['goal', 'war', '<UNK>', 'issue']]
[['gettysburg'], ['union', 'vi